In [12]:
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import pandas as pd

In [30]:
colors=["blue","brown","green","teal","pink","orange","black","red","yellow","magenta","black"]
cmap={mcolors.to_hex(c):c for c in colors}
taxonomy_legend_to_color={'Bacteria': 'blue',
 'Fungi': 'brown',
 'Viridiplantae': 'green',
 'Sar': 'teal',
 'Viruses': 'yellow',
 'Non-Arthropod Metazoa': 'orange',
 'Arthropod (non-chimera)': 'pink',
 'Arthropod chimera (primary)': 'magenta',
 'Arthropod chimera (secondary)': 'red',
 'Other': 'black'}
color_to_taxonomy_legend={
    "blue": "Bacteria",
    "brown": "Fungi",
    "green":"Viridiplantae",
    "teal":"Sar",
    "yellow" : "Viruses",
    "orange": "Non-Arthropod Metazoa",
    "pink": "Arthropod (non-chimera)",
    "magenta" : "Arthropod chimera (primary)",
    "red": "Arthropod chimera (secondary)",
    "black": "Other"
    
}

In [7]:
def add_legend(x):
    
    file_path=f"data_upload_3_24_2025/chimera_phylogenetic_datasets/{x}/itol_annotation_colorstrip.txt"
    f=open(file_path,"r").readlines()
    if "LEGEND_TITLE Taxonomic labels\n" not in f:
        
        colorsi=list(set(dftax['color']))

        cmapi={x:color_to_taxonomy_legend[cmap[x]] for x in colorsi}
        cmapi = dict(sorted(cmapi.items(), key=lambda item: item[1]))
        legend_text="LEGEND_TITLE Taxonomic labels\n"
        shapes="LEGEND_SHAPES "
        for i in range(len(cmapi)):
            shapes+=("1 ")
        shapes=shapes[:-1]
        legend_text=legend_text+"\n"+shapes
        legend_colors="LEGEND_COLORS "
        for c in cmapi:
            legend_colors+=c+" "
        legend_colors=legend_colors[:-1]
        legend_text=legend_text+"\n"+legend_colors
        legend_LABELS="LEGEND_LABELS "
        for c in cmapi:
            legend_LABELS+=cmapi[c].replace(" ","_")+" "
        legend_LABELS=legend_LABELS[:-1]
        legend_text=legend_text+"\n"+legend_LABELS+"\n"+"\n"
        legend_text=legend_text+"STRIP_WIDTH 40\n\n"
        with open(file_path, "r") as file:
            lines = file.readlines()

        for i, line in enumerate(lines):
            if line.strip() == "DATA":
                insert_index = i
                break
        lines.insert(insert_index, legend_text)
        with open(file_path, "w") as file:
            file.writelines(lines)
    else:
        print("present")
    

In [32]:
def upload(n,project,workspace):
    
    test = Itol()
    
    try:
        current_dir = Path(os.path.realpath(f"{directory}/{n}/MAD_rooted_iqtree_result.treefile")).parent
        tree = current_dir/"MAD_rooted_iqtree_result.treefile"
        test.add_file(tree)
    except:
        
        current_dir = Path(os.path.realpath(f"{directory}/{n}/iqtree_result.treefile")).parent
        tree = current_dir/"iqtree_result.treefile"
        test.add_file(tree)
        print(n)
   
    
    root_path = current_dir.parent
    sys.path.append(str(root_path))
    

    # Set the tree file


    
    test.add_file(current_dir / 'itol_annotation_colorstrip.txt')
    test.add_file(current_dir / 'itol_annotation_taxonomic.txt')
    test.add_file(current_dir / 'itol_annotation_sequence_description.txt')
    # Add parameters

    test.params['APIkey'] = 'xZMZCofEGnKYSfuydC0PaQ'
    test.params['treeName'] = n
    test.params['projectName'] = project
    test.params['workspaceName'] = workspace
  
    # Submit the tree
    good_upload = test.upload()
    if not good_upload:
        pprint(n)
        print('There was an error:' + test.comm.upload_output)
        sys.exit(1)
 

In [33]:
def upload_Wrapper(ps):
    n, project, workspace = ps
    add_legend(n)
    upload(n,project,workspace)


In [34]:
import os
directory="data_upload_3_24_2025/chimera_phylogenetic_datasets"

In [35]:
td=[x for x in os.listdir(f"{directory}") if "Meta" in x or "HGT" in x]
with mp.Pool(30) as p:
    results = p.map(upload_Wrapper, [[n,"Arthropod_chimeric_HGT_trees","Arthropod_chimeric_HGT_2024"] for n in td])